# 手写数字识别 - DL - 神经网络

## The first demo about Keras

### 若遇到梯子问题：
#### [【问题解决方案】Keras手写数字识别-ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接](https://www.cnblogs.com/anliux/p/10793202.html)

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.datasets import mnist
# from load_data import *
# categorical_crossentropy

# #clean data
# 这里应该是预处理数据，导入数据由那条import负责
# 细微修改
def load_data(): # 原名字被载入函数占了，这里末尾加上set以区别
    # 下面这条不能用了，改为新的
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # x_train, y_train = load_train_images(), load_train_labels()
    # x_test, y_test = load_test_images(), load_test_labels()
    
    number = 10000 # 由10000改为60000，可能数据量不同
        # 官网下载的训练集包含60000个示例，测试集包含10000个示例。
    
    # 下面好多语句都可以合并为一句，这里不重构了
    x_train = x_train[0:number]
    y_train = y_train[0:number]
    x_train = x_train.reshape(number, 28*28)
    x_test = x_test.reshape(x_test.shape[0],28*28)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # convert class vectors to binary class matrics
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    # 这两句是个啥，没啥用，注释掉
    # x_train = x_train
    # x_test = x_test
    # x_test = np.random.normal(x_test)
    x_train = x_train/255
    x_test = x_test/255
    return(x_train, y_train),(x_test, y_test)

# 载入数据
(x_train, y_train),(x_test, y_test) = load_data() # 预处理函数改名字了，这里也改：data-->dataset

In [7]:
# 测试一下此时x_train矩阵的形状
x_train.shape

# 运行后需要先下载数据，需要科学上网否则会下载失败...
# Downloading data from https://s3.amazonaws.com/img-datasets/mnist.npz

(10000, 784)

In [11]:
# 老师课上demo的代码-原始人版本

# 选model
model = Sequential()

# 搭建神经网络
model.add(Dense(input_dim=28*28, units=633, activation='sigmoid'))
model.add(Dense(units=633, activation='sigmoid'))
model.add(Dense(units=633, activation='sigmoid'))
model.add(Dense(units=10, activation='softmax'))

# 自选loss Function等参数
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

# train模型
model.fit(x_train, y_train, batch_size=100, epochs=20)

# 测试结果，并打印accuracy
result = model.evaluate(x_test, y_test)
print('\nTest loss:', result[0])
print('\nAccuracy:', result[1])

Epoch 1/20
10000/10000 [==============================] - 4s 385us/step - loss: 0.0915 - acc: 0.1078
Epoch 2/20
10000/10000 [==============================] - 3s 255us/step - loss: 0.0900 - acc: 0.1027
Epoch 3/20
10000/10000 [==============================] - 2s 193us/step - loss: 0.0900 - acc: 0.1058
Epoch 4/20
10000/10000 [==============================] - 3s 331us/step - loss: 0.0900 - acc: 0.1118
Epoch 5/20
10000/10000 [==============================] - 2s 185us/step - loss: 0.0899 - acc: 0.1088
Epoch 6/20
10000/10000 [==============================] - 2s 187us/step - loss: 0.0899 - acc: 0.1125
Epoch 7/20
10000/10000 [==============================] - 2s 183us/step - loss: 0.0899 - acc: 0.1100
Epoch 8/20
10000/10000 [==============================] - 2s 186us/step - loss: 0.0899 - acc: 0.1111
Epoch 9/20
10000/10000 [==============================] - 2s 184us/step - loss: 0.0899 - acc: 0.1085
Epoch 10/20
10000/10000 [==============================] - 2s 188us/step - loss: 0.0899 - a

In [17]:
# 调参后版本
# 改动地方主要为： 
# - 激励函数由sigmoid改为relu 
# - loss function由mse改为categorical_crossentropy 
# - 增加了Dropout，防止过拟合

# 选model
model = Sequential()

# 搭建神经网络
# 改动：4点
# 1-中间层units由633改为700
# 2-激活函数由sigmoid改为relu
# 3-原四个Dense，删去一个中间层，只留三个Dense
# 4-在三个Dense的每两个Dense中间加入Dropout
# batch-epochs=100,20时，三Dense好于四Dense
# batch-epochs=1,2时，三Dense好于四Dense
model.add(Dense(input_dim=28*28, units=700, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=700, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation='softmax'))

# 自选loss Function等参数
# 改动：1点
# 1-loss function由mse改为categorical_crossentropy
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

# train模型
# 改动：1点
# 1-参数列表最后加一个validation_split（交叉验证？）
# 2-batch-size=1, epochs=2时，acc变为0.9314(3个Dense)，0.9212（4个Dense）
model.fit(x_train, y_train, batch_size=1, epochs=2, validation_split=0.05)

# 测试结果，并打印accuracy
result = model.evaluate(x_test, y_test)
print('\nTest loss:', result[0])
print('\nAccuracy:', result[1])

Train on 9500 samples, validate on 500 samples
Epoch 1/2
9500/9500 [==============================] - 115s 12ms/step - loss: 0.0247 - acc: 0.8318 - val_loss: 0.0118 - val_acc: 0.9220
Epoch 2/2
10000/10000 [==============================] - 1s 63us/step

Test loss: 0.01238522016274801

Accuracy: 0.9178
